# vLLM + DeepSpeed

```

┌───────────────────────────────────────────────────────────────┐
│                OUTER LOOP: RL iterations / epochs             │
│                                                               │
│   ┌───────────────┐      ┌───────────────────┐      ┌────────┐│
│   │     vLLM      │ ───▶ │  Compute Rewards  │ ───▶ │DeepSpeed││
│   │ Generate      │      │ (human/RM via     │      │ Train   ││
│   │ Trajectories  │      │  vLLM if needed)  │      │ Policy   ││
│   └──────┬────────┘      └─────────┬─────────┘      └────┬───┘│
│          │                          │                    │    │
│          └──────────────────────────┴────────────────────┘    │
│                     (Updated Policy fed back)                 │
└───────────────────────────────────────────────────────────────┘

```

## Motivation
Imagine a reinforcement learning (RL) training loop: we **generate trajectories**, compute rewards, and **update the policy** model.  
**vLLM** and **DeepSpeed** act as *system-level optimizers* for these two core stages.  
vLLM accelerates **trajectory generation** and **reward inference**, making large-scale sampling highly efficient through optimized GPU memory management (PagedAttention).  
DeepSpeed accelerates **policy training**, enabling distributed, memory-efficient optimization (via ZeRO) for massive models.  
Together, they form the backbone of a scalable RL pipeline — vLLM for fast generation, DeepSpeed for efficient learning — both maximizing GPU utilization across the loop.





## 🧠 Section 1 — The KV Cache and Why It Matters


### Attention Mechanism Formula

The general form of the **scaled dot-product attention** is:

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

Where:

- $ Q $ = Query matrix  
- $ K $ = Key matrix  
- $ V $ = Value matrix  
- $ d_k $ = Dimension of the key vectors (used for scaling)

At each decoding step, the model must compute attention between the **current query** and **all past key–value pairs** to determine what context is most relevant.

---

### 2. Computational Cost Without Caching

During **autoregressive generation**, each new token requires recomputing the attention using *all* previous tokens:

$$
\text{Attention}_t = \text{softmax}\left(\frac{Q_t K_{1:t}^T}{\sqrt{d_k}}\right)V_{1:t}
$$

That means for every new token $ t $, you rebuild $ K_{1:t} $ and $ V_{1:t} $ — leading to **quadratic time complexity** $ O(T^2) $ for a sequence of length $ T $.

This becomes very expensive for long sequences.
```

            Generating the 4th token

   ┌───────┬───────┬───────┐
   │ x1    │ x2    │ x3    │
   └──┬────┴──┬────┴──┬────┘
      │       │       │
      ▼       ▼       ▼
     K1,V1   K2,V2   K3,V3      ← use Keys/Values from ALL previous tokens

              ▲
              │
              │  Query from LAST hidden state (x3)
              │
             Q4

   Attention: Q4 attends over {K1,K2,K3}
   ↓
   Produces next representation → predict token 4

```
---

### 3. Introducing the KV Cache

To avoid recomputing the same $ K $ and $ V $ at every step, we **cache** them:

$$
K_{1:t} = [K_{1:t-1}; K_t], \quad V_{1:t} = [V_{1:t-1}; V_t]
$$

At each new step:
- The model **reuses** the previously stored keys and values.
- Only the **new** $ K_t $ and $ V_t $ for the latest token are computed and appended to the cache.

This allows attention to be computed efficiently as:

$$
\text{Attention}_t = \text{softmax}\left(\frac{Q_t K_{\text{cache}}^T}{\sqrt{d_k}}\right)V_{\text{cache}}
$$

---

### 4. Resulting Benefits

- ✅ **Reduces computation** — only one step of attention per new token.
- ✅ **Reduces memory access** — no need to rebuild past hidden states.
- ✅ **Enables fast autoregressive decoding**, making real-time generation (like chat or translation) feasible.

---

### 5. Summary Table

| Step | Without KV Cache | With KV Cache |
|------|------------------|---------------|
| Compute Keys/Values | Recomputed each step | Reused from cache |
| Complexity | $ O(T^2) $ | $ O(T) $ |
| Speed | Slow for long sequences | Much faster |


### 6. Throughput via Batching, and Its Two Problems
To use the GPU efficiently, we **batch** requests. But in practice:
1) **Asynchronous arrivals:** requests don’t start together.  
2) **Variable lengths:** prompts/outputs differ; some finish early.

This leads to idle threads and poor packing unless we **rebatch every iteration** (a.k.a. *iteration-level scheduling* / *cellular batching*).

### 7. KV Cache Still Has Issues
Even with better batching, many concurrent, variable-length requests cause **KV cache memory fragmentation**, turning into a GPU memory management problem that Section 2 addresses.



## ⚙️ Section 2 — PagedAttention: Solving the KV Cache Memory Problem

### 1. Motivation: KV Cache Fixes Computation, but Creates a Memory Challenge
KV caching saves compute but each request’s cache **grows autoregressively** as new tokens are generated. With many concurrent requests, caches grow and finish at different times, stressing GPU memory.

### 2. The Naïve Way: Consecutive (Contiguous) Allocation
```
┌───────────────────────────────────────────────┐
│ [Req A (50 tok)] [Req B (30 tok)] [Req C (80)]│
└───────────────────────────────────────────────┘
```
```
┌──────────────────────────────────────────────────────────────┐
│ [Req A (50 tok)] [cleared (30 tok)] [Req C (80)] [Req D (60)]│
└──────────────────────────────────────────────────────────────┘
```
Each request holds one **contiguous** KV tensor (≈ `[layers, heads, tokens, head_dim]`). When **A** generates the next token, it must append immediately after its block — but **B** is already there. Growing A would require shifting/copying huge tensors and stalling kernels.

Finished requests create **holes** that are often unusable for new requests → **fragmentation**.

### 3. The “Cut It Up” Idea — Why It’s Hard
Splitting a request’s KV into smaller chunks would pack memory better, but then the attention kernel must know **where every token’s K/V lives** and reconstruct the correct order — hard to track without structure.
```
┌───────────────────────────────────────────────┐
│ [Req A (50 tok)] [Req B (30 tok)] [Req C (80)]│
└───────────────────────────────────────────────┘
```
```
┌──────────────────────────────────────────────────────────────────────────┐
│ [Req A (50 tok)] [Req D part 1 (30 tok)] [Req C (80)] [Req D part 2 (30)]│
└──────────────────────────────────────────────────────────────────────────┘
```

### 4. The Breakthrough: PagedAttention
**PagedAttention** slices GPU memory into **uniform pages** (small fixed-size blocks, e.g., 16 tokens per page). Each request’s KV cache is a list of pages, not one big block.

```
GPU pages:  P1 | P2 | P3 | P4 | P5 | P6 | ...
Req A  → P1,P2
Req B  → P3
Req C  → P4,P5
A grows → take P6 (no moves, just add a page)
```

### 5. The Page Table (Indirection)
A lightweight **page table** maps *logical token indices* → *(page, offset)*:
```
token 37 → page_table[37//page_size] + (37 % page_size)
```
Attention kernels read K/V via this mapping, so KV can be physically scattered yet **logically contiguous**.

### 6. Why This Optimizes Memory Use
- **No fragmentation:** freed pages are uniform and immediately reusable
- **No large copies:** growing requests just add pages; finishing requests return pages
- **Concurrency-friendly:** requests grow/finish independently
- **Virtual-memory-like behavior:** logical order preserved via page table

### 7. Summary Visualization
```
Without paging (contiguous): AAAAAA BBB … CCCCC D …
→ gaps, blocked growth, relocations

With PagedAttention: P1(A) P2(A) P3(B) P4(C) P5(Anew) …
→ reuse pages, no relocations, steady throughput
```



## 🏗️ Section 3 — Inside vLLM: How Paging and Scheduling Work Together

### 1. Goal
Keep the GPU highly utilized despite **asynchronous arrivals** and **variable-length** requests.

### 2. Runtime Pipeline
```
User/API → Request Queue → Scheduler (per-iteration) → GPU Workers → PagedAttention → Streams Out
```

- **Request Queue:** holds incoming work
- **Scheduler:** mixes new and ongoing requests instead of waiting for all to finish before starting new ones. It collects all active sequences that need their next token. It admits new requests if there’s free KV-cache memory. It removes finished sequences (those that reached EOS or max length).
- **GPU Workers:** run attention kernels; read K/V via page table; write new K/V into free pages
- **PagedAttention:** provides flexible KV memory without copies
- **Streaming:** return tokens as they’re ready while continuing decoding

### 3. Why This Works
- PagedAttention allows **dynamic batching** without moving memory
- The scheduler keeps batches full each iteration
- Memory stays compact and reusable; fewer OOMs and higher throughput

### 4. Concept Diagram
```
┌──────────────────────────────────────────────────────────────────────────┐
│ [Incoming Requests] → Queue → Scheduler → GPU Worker → PagedAttention →  │
│                             ↑                                   ↓        │
│                         Stream tokens back to clients continuously       │
└──────────────────────────────────────────────────────────────────────────┘
```



## 🚀 Section 4 — Minimal “How to Run” Demos



### 4.1 Install

In [ ]:

# Fresh environment recommended
# If you're in Jupyter, the leading '%' is allowed; in plain Python, remove it.
%pip install -U vllm transformers accelerate openai



### 4.2 Pick a model
`Qwen3-4B-Instruct-2507`


In [ ]:
from vllm import LLM, SamplingParams
import torch

# Check GPU
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Load model with simpler configuration
model = "/gpfs/radev/project/zhuoran_yang/xh338/models/Qwen3-4B-Instruct-2507"

# Try with minimal settings
llm = LLM(
    model=model,
    dtype="float16",
    gpu_memory_utilization=0.9,
    max_model_len=2048,  # Reduce max length
    enforce_eager=True   # Disable compilation
)

# Test with simple prompts
prompts = [
    "<|im_start|>user\nHi, how are you?<|im_end|>\n<|im_start|>assistant\n",
    "<|im_start|>user\nTell me a short story about a robot.<|im_end|>\n<|im_start|>assistant\n",
    "<|im_start|>user\nExplain what AI is in simple terms.<|im_end|>\n<|im_start|>assistant\n"
]

params = SamplingParams(
    temperature=0.7,
    top_p=0.9,
    max_tokens=200,
    skip_special_tokens=False
)

outputs = llm.generate(prompts, params)

for i, output in enumerate(outputs):
    print(f"\n{'='*50}")
    response = output.outputs[0].text
    print(f"Response {i+1}: {response}")